## V1 — City Size Classes

**Goal:** Urban hierarchy.
**Changes:** Assign `class ∈ {large, medium, small}` with quotas (e.g., 10/30/60%). Distinct lognormal per class.
**Validation:** Class counts match; medians ordered `large > medium > small`.

In [14]:
"""
Jupyter notebook cell — Version 1 (Nodes only)
Probabilistic city classes + uniform population ranges per class.
- Placement: uniform within bbox (same as V0)
- Class assignment: per-node categorical draw with probabilities
- Population: uniform integer in [min, max] for the node’s class
- Outputs: nodes.csv, meta.json, preview.png (color = population; marker shape = class)

Usage: run this cell. Edit `V1Config` to change class probabilities or ranges.
"""
from __future__ import annotations

import json
import os
import time
import hashlib
from dataclasses import dataclass, field
from typing import Tuple, Dict, Any, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from matplotlib.lines import Line2D


# ------------------------------
# Config
# ------------------------------
@dataclass
class V1Config:
    seed: int = 42
    n_nodes: int = 120
    bbox_km: Tuple[float, float, float, float] = (0.0, 0.0, 200.0, 200.0)  # (minx, miny, maxx, maxy)

    # Per-class probabilities (will be normalized to sum 1). Each node draws its class independently.
    class_prob: Dict[str, float] = field(
        default_factory=lambda: {"large": 0.10, "medium": 0.30, "small": 0.60}
    )

    # Uniform population ranges per class (inclusive of min/max)
    class_ranges: Dict[str, Dict[str, int]] = field(
        default_factory=lambda: {
            # Choose non-overlapping ranges so medians satisfy large > medium > small
            "large":  {"min": 600_000, "max": 1_500_000},
            "medium": {"min": 150_000, "max":   500_000},
            "small":  {"min":   1_000, "max":   120_000},
        }
    )

    # Output & metadata
    out_dir: str = "maps/sv1.1/dv0.2_v1_classes_prob_uniform"
    crs: str = "EPSG:3857"  # Synthetic planar; coordinates stored in km for simplicity
    schema_version: str = "1.1"  # still includes optional `class` column
    dataset_version: str = "0.2"


# ------------------------------
# Core helpers
# ------------------------------

def set_seed(seed: int) -> None:
    np.random.seed(seed)


def generate_positions_uniform(bbox_km: Tuple[float, float, float, float], n: int) -> np.ndarray:
    minx, miny, maxx, maxy = bbox_km
    if not (minx < maxx and miny < maxy):
        raise ValueError("Invalid bbox: must satisfy minx<maxx and miny<maxy")
    xs = np.random.uniform(minx, maxx, size=n)
    ys = np.random.uniform(miny, maxy, size=n)
    return np.column_stack([xs, ys])


def _sample_classes(n: int, class_prob: Dict[str, float]) -> List[str]:
    labels = list(class_prob.keys())
    probs = np.array([class_prob[k] for k in labels], dtype=float)
    probs = probs / probs.sum()
    draws = np.random.choice(labels, size=n, p=probs)
    return draws.tolist()


def _sample_uniform_int(low: int, high: int, size: int) -> np.ndarray:
    """Inclusive uniform integer sampling in [low, high]."""
    if high < low:
        raise ValueError(f"Invalid range: [{low},{high}]")
    return np.random.randint(low, high + 1, size=size, dtype=int)


def generate_nodes_v1(cfg: V1Config) -> pd.DataFrame:
    # positions
    pts = generate_positions_uniform(cfg.bbox_km, cfg.n_nodes)

    # classes (probabilistic per-node)
    classes = _sample_classes(cfg.n_nodes, cfg.class_prob)

    # populations per class (uniform within class range)
    pops = np.empty(cfg.n_nodes, dtype=int)
    for cls in cfg.class_prob.keys():
        idx = [i for i, c in enumerate(classes) if c == cls]
        if not idx:
            continue
        r = cfg.class_ranges.get(cls, None)
        if r is None:
            raise KeyError(f"Missing class range for '{cls}'")
        pops[idx] = _sample_uniform_int(int(r["min"]), int(r["max"]), size=len(idx))

    df = pd.DataFrame({
        "id": np.arange(cfg.n_nodes, dtype=int),
        "x_km": pts[:, 0],
        "y_km": pts[:, 1],
        "class": classes,
        "pop": pops,
    })
    return df


def validate_nodes(df: pd.DataFrame, cfg: V1Config) -> Dict[str, Any]:
    minx, miny, maxx, maxy = cfg.bbox_km
    metrics: Dict[str, Any] = {}

    # Count
    n = len(df)
    if n != cfg.n_nodes:
        raise AssertionError(f"Node count mismatch: expected {cfg.n_nodes}, got {n}")
    metrics["n_nodes"] = n

    # Bounds
    inside_x = (df["x_km"] >= minx) & (df["x_km"] <= maxx)
    inside_y = (df["y_km"] >= miny) & (df["y_km"] <= maxy)
    violations = int((~(inside_x & inside_y)).sum())
    if violations:
        raise AssertionError(f"{violations} nodes fall outside bbox")
    metrics["bbox"] = {"minx": minx, "miny": miny, "maxx": maxx, "maxy": maxy}

    # Class counts (no strict quota check; just report)
    metrics["class_counts"] = df["class"].value_counts().to_dict()

    # Population summaries and median ordering (should hold if ranges are non-overlapping)
    med = df.groupby("class")["pop"].median().to_dict()
    metrics["class_medians"] = {k: int(v) for k, v in med.items()}
    try:
        if not (med["large"] > med["medium"] > med["small"]):
            # Don’t hard fail, just record a flag
            metrics["median_order_ok"] = False
        else:
            metrics["median_order_ok"] = True
    except KeyError:
        metrics["median_order_ok"] = False

    # Global population range
    pmin, pmax = int(df["pop"].min()), int(df["pop"].max())
    metrics["pop_range_observed"] = {"min": pmin, "max": pmax}
    metrics["pop_percentiles"] = {q: int(np.percentile(df["pop"], q)) for q in (5, 25, 50, 75, 95)}

    return metrics


def preview_nodes(df: pd.DataFrame, cfg: V1Config, save_path: str) -> None:
    """Scatter sized by population (color = population, colorbar legend).
    Marker shape encodes class ({large: square, medium: triangle, small: circle}).
    Annotates the top-3 most populated cities with population labels.
    """
    minx, miny, maxx, maxy = cfg.bbox_km

    vmax = df["pop"].max()
    vmin = df["pop"].min()

    markers = {"large": "s", "medium": "^", "small": "o"}

    plt.figure(figsize=(6, 6))

    sc = None
    for cls in ["large", "medium", "small"]:
        sub = df[df["class"] == cls]
        if sub.empty:
            continue
        sc = plt.scatter(
            sub["x_km"],
            sub["y_km"],
            s=10 + 90 * np.sqrt(sub["pop"].values / vmax),
            c=sub["pop"].values.astype(float),
            vmin=vmin,
            vmax=vmax,
            marker=markers.get(cls, "o"),
            label=cls.title(),
        )

    if sc is not None:
        cbar = plt.colorbar(sc)
        cbar.set_label("Population")
        try:
            cbar.ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
        except Exception:
            pass

    handles = [Line2D([], [], marker=markers.get(cls, "o"), linestyle="None", label=cls.title())
               for cls in ["large", "medium", "small"]]
    plt.legend(handles=handles, title="Class", loc="best", framealpha=0.8)

    # Annotate top-3 by population
    top3 = df.nlargest(3, "pop").copy()
    dx = 0.01 * (maxx - minx)
    dy = 0.01 * (maxy - miny)
    for _, row in top3.iterrows():
        label = f"{int(row['pop']):,}"
        plt.text(
            row["x_km"] + dx,
            row["y_km"] + dy,
            label,
            fontsize=8,
            ha="left",
            va="bottom",
            bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7),
        )

    plt.title("Nodes — V1 (probabilistic classes; uniform ranges)")
    plt.xlabel("x (km)")
    plt.ylabel("y (km)")
    plt.xlim(minx, maxx)
    plt.ylim(miny, maxy)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()


def compute_metrics_hash(metrics: Dict[str, Any]) -> str:
    blob = json.dumps(metrics, sort_keys=True).encode("utf-8")
    return hashlib.sha256(blob).hexdigest()[:16]


def save_artifacts(df: pd.DataFrame, cfg: V1Config, metrics: Dict[str, Any]) -> Dict[str, str]:
    os.makedirs(cfg.out_dir, exist_ok=True)

    nodes_path = os.path.join(cfg.out_dir, "nodes.csv")
    preview_path = os.path.join(cfg.out_dir, "preview.png")
    meta_path = os.path.join(cfg.out_dir, "meta.json")

    df.to_csv(nodes_path, index=False)
    preview_nodes(df, cfg, preview_path)

    meta = {
        "schema_version": cfg.schema_version,
        "dataset_version": cfg.dataset_version,
        "crs": cfg.crs,
        "seed": cfg.seed,
        "generator": {
            "name": "nodes_v1_classes_prob_uniform",
            "params": {
                "n_nodes": cfg.n_nodes,
                "bbox_km": cfg.bbox_km,
                "class_prob": cfg.class_prob,
                "class_ranges": cfg.class_ranges,
            },
        },
        "region_bbox": list(cfg.bbox_km),
        "metrics": metrics,
        "metrics_hash": compute_metrics_hash(metrics),
        "created_at_utc": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
    }
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

    return {"nodes": nodes_path, "preview": preview_path, "meta": meta_path}


# ------------------------------
# Orchestration
# ------------------------------

def main(cfg: V1Config | None = None) -> pd.DataFrame:
    cfg = cfg or V1Config()
    set_seed(cfg.seed)

    df = generate_nodes_v1(cfg)
    metrics = validate_nodes(df, cfg)
    paths = save_artifacts(df, cfg, metrics)

    print("\n[Nodes V1] Build complete:\n" + "-" * 40)
    print(f"Nodes: {len(df)} | bbox: {cfg.bbox_km}")
    print(f"Class counts: {metrics['class_counts']}")
    print(f"Class medians: {metrics['class_medians']} (order ok = {metrics.get('median_order_ok')})")
    print(f"Saved: nodes → {paths['nodes']}\n       preview → {paths['preview']}\n       meta → {paths['meta']}")
    print(f"Metrics hash: {compute_metrics_hash(metrics)}")
    return df


# ------------------------------
# Run (notebook-friendly)
# ------------------------------
_cfg = V1Config(
    seed=30,
    n_nodes=30,
    bbox_km=(0.0, 0.0, 200.0, 200.0),
    class_prob={"large": 0.10, "medium": 0.30, "small": 0.60},
    class_ranges={
        "large":  {"min": 600_000, "max": 1_500_000},
        "medium": {"min": 100_000, "max":   300_000},
        "small":  {"min":   1_000, "max":   60_000},
    },
    out_dir="maps/sv1.1/dv0.2_v1_classes",
)

_ = main(_cfg)



[Nodes V1] Build complete:
----------------------------------------
Nodes: 30 | bbox: (0.0, 0.0, 200.0, 200.0)
Class counts: {'small': 17, 'medium': 11, 'large': 2}
Class medians: {'large': 1296607, 'medium': 215837, 'small': 43342} (order ok = True)
Saved: nodes → maps/sv1.1/dv0.2_v1_classes\nodes.csv
       preview → maps/sv1.1/dv0.2_v1_classes\preview.png
       meta → maps/sv1.1/dv0.2_v1_classes\meta.json
Metrics hash: af5893ff0c1a3f51


### V1.1 — Single-Core Density Field

**Goal:** Spatial realism with a capital-like core.
**Changes:** Rejection sampling from a 2D Gaussian density. Parameters: `core_location`, `core_sigma_km`, `density_strength`.
**Outputs:** (optional) `core_dist_km`.
**Validation:** Mean distance to core below uniform baseline.

In [18]:
"""
Jupyter notebook cell — Version 1.1 (Nodes only)
Single-core density field with rejection/mix sampling + uniform populations.
- Placement: mixture sampling — with probability `density_strength`, draw from a 2D Gaussian
  centered at `core_location` (sigma = `core_sigma_km`), otherwise uniform in bbox.
  Points falling outside the bbox are resampled (up to `max_gauss_resamples` per draw).
- Population: uniform integer in [pop_min, pop_max]
- Outputs: nodes.csv, meta.json, preview.png (color = population, top-3 annotated)
- Extra column: `core_dist_km` (distance to the core)
- Validation: mean distance to core must be **below** a Monte‑Carlo uniform baseline.

Usage (single cell): run as-is or tweak `V11Config`.
"""
from __future__ import annotations

import json
import os
import time
import hashlib
from dataclasses import dataclass
from typing import Tuple, Dict, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter


# ------------------------------
# Config
# ------------------------------
@dataclass
class V11Config:
    seed: int = 42
    n_nodes: int = 120
    bbox_km: Tuple[float, float, float, float] = (0.0, 0.0, 200.0, 200.0)  # (minx, miny, maxx, maxy)

    # Density field parameters
    core_location: Tuple[float, float] | None = None  # (cx, cy). If None, use bbox center
    core_sigma_km: float = 25.0                       # std dev of the Gaussian (km)
    density_strength: float = 0.8                     # probability to sample from the Gaussian (0..1)
    max_gauss_resamples: int = 200                    # per-point attempts if Gaussian falls outside bbox

    # Population (simple, uniform like V0)
    pop_min: int = 1_000
    pop_max: int = 1_000_000

    # Validation
    baseline_samples: int = 5000                      # Monte‑Carlo size for uniform baseline
    mean_dist_margin_km: float = 0.5                  # require mean_dist <= baseline_mean - margin

    # Output & metadata
    out_dir: str = "maps/sv1.1/dv0.3_v1_1_single_core"
    crs: str = "EPSG:3857"
    schema_version: str = "1.1"  # minor: extra optional column `core_dist_km`
    dataset_version: str = "0.3"


# ------------------------------
# Helpers
# ------------------------------

def set_seed(seed: int) -> None:
    np.random.seed(seed)


def _bbox_center(bbox):
    minx, miny, maxx, maxy = bbox
    return (0.5 * (minx + maxx), 0.5 * (miny + maxy))


def _in_bounds(x: float, y: float, bbox) -> bool:
    minx, miny, maxx, maxy = bbox
    return (minx <= x <= maxx) and (miny <= y <= maxy)


def _sample_point_gauss(cx: float, cy: float, sigma: float, bbox, rng: np.random.RandomState, max_tries: int) -> Tuple[float, float] | None:
    for _ in range(max_tries):
        x = cx + sigma * rng.randn()
        y = cy + sigma * rng.randn()
        if _in_bounds(x, y, bbox):
            return (x, y)
    return None


def generate_positions_single_core(cfg: V11Config, rng: np.random.RandomState) -> np.ndarray:
    minx, miny, maxx, maxy = cfg.bbox_km
    if not (minx < maxx and miny < maxy):
        raise ValueError("Invalid bbox: must satisfy minx<maxx and miny<maxy")

    cx, cy = cfg.core_location or _bbox_center(cfg.bbox_km)
    if not _in_bounds(cx, cy, cfg.bbox_km):
        raise ValueError("core_location must lie within bbox")

    strength = float(cfg.density_strength)
    if not (0.0 <= strength <= 1.0):
        raise ValueError("density_strength must be in [0,1]")

    pts = np.empty((cfg.n_nodes, 2), dtype=float)
    for i in range(cfg.n_nodes):
        if rng.rand() < strength:
            pt = _sample_point_gauss(cx, cy, cfg.core_sigma_km, cfg.bbox_km, rng, cfg.max_gauss_resamples)
            if pt is None:
                # Fallback to uniform if Gaussian keeps falling outside
                x = rng.uniform(minx, maxx)
                y = rng.uniform(miny, maxy)
                pts[i] = (x, y)
            else:
                pts[i] = pt
        else:
            x = rng.uniform(minx, maxx)
            y = rng.uniform(miny, maxy)
            pts[i] = (x, y)
    return pts


def generate_nodes_v11(cfg: V11Config) -> pd.DataFrame:
    rng = np.random.RandomState(cfg.seed)
    pts = generate_positions_single_core(cfg, rng)

    # Uniform population per node
    pops = rng.randint(cfg.pop_min, cfg.pop_max + 1, size=cfg.n_nodes).astype(int)

    cx, cy = cfg.core_location or _bbox_center(cfg.bbox_km)
    core_dist = np.sqrt((pts[:, 0] - cx) ** 2 + (pts[:, 1] - cy) ** 2)

    df = pd.DataFrame({
        "id": np.arange(cfg.n_nodes, dtype=int),
        "x_km": pts[:, 0],
        "y_km": pts[:, 1],
        "pop": pops,
        "core_dist_km": core_dist,
    })
    return df


def _mean_distance_to_core_uniform_baseline(cfg: V11Config, rng: np.random.RandomState) -> float:
    minx, miny, maxx, maxy = cfg.bbox_km
    cx, cy = cfg.core_location or _bbox_center(cfg.bbox_km)
    xs = rng.uniform(minx, maxx, size=cfg.baseline_samples)
    ys = rng.uniform(miny, maxy, size=cfg.baseline_samples)
    d = np.sqrt((xs - cx) ** 2 + (ys - cy) ** 2)
    return float(d.mean())


def validate_nodes(df: pd.DataFrame, cfg: V11Config) -> Dict[str, Any]:
    minx, miny, maxx, maxy = cfg.bbox_km
    cx, cy = cfg.core_location or _bbox_center(cfg.bbox_km)

    metrics: Dict[str, Any] = {}

    # Count
    n = len(df)
    if n != cfg.n_nodes:
        raise AssertionError(f"Node count mismatch: expected {cfg.n_nodes}, got {n}")
    metrics["n_nodes"] = n

    # Bounds
    inside_x = (df["x_km"] >= minx) & (df["x_km"] <= maxx)
    inside_y = (df["y_km"] >= miny) & (df["y_km"] <= maxy)
    violations = int((~(inside_x & inside_y)).sum())
    if violations:
        raise AssertionError(f"{violations} nodes fall outside bbox")
    metrics["bbox"] = {"minx": minx, "miny": miny, "maxx": maxx, "maxy": maxy}

    # Mean distance to core vs baseline
    mean_dist = float(df["core_dist_km"].mean())
    baseline_rng = np.random.RandomState(cfg.seed + 1)
    baseline_mean = _mean_distance_to_core_uniform_baseline(cfg, baseline_rng)

    if not (mean_dist <= baseline_mean - cfg.mean_dist_margin_km):
        raise AssertionError(
            f"Mean distance to core {mean_dist:.2f} km is not significantly below uniform baseline {baseline_mean:.2f} km"
        )
    metrics["mean_core_dist_km"] = mean_dist
    metrics["baseline_mean_core_dist_km"] = baseline_mean
    metrics["improvement_km"] = baseline_mean - mean_dist

    # Population summaries
    pmin, pmax = int(df["pop"].min()), int(df["pop"].max())
    if pmin < cfg.pop_min or pmax > cfg.pop_max:
        raise AssertionError(
            f"Population out of range: observed [{pmin},{pmax}] vs cfg [{cfg.pop_min},{cfg.pop_max}]"
        )
    metrics["pop_range_observed"] = {"min": pmin, "max": pmax}
    metrics["pop_percentiles"] = {q: int(np.percentile(df["pop"], q)) for q in (5, 25, 50, 75, 95)}

    return metrics


def preview_nodes(df: pd.DataFrame, cfg: V11Config, save_path: str) -> None:
    """Scatter sized by population with color gradient and colorbar legend.
    Annotates the top-3 most populated cities with their populations.
    """
    minx, miny, maxx, maxy = cfg.bbox_km
    cx, cy = cfg.core_location or _bbox_center(cfg.bbox_km)

    sizes = 10 + 90 * np.sqrt(df["pop"].values / df["pop"].max())
    pop_vals = df["pop"].values.astype(float)

    plt.figure(figsize=(6, 6))
    sc = plt.scatter(df["x_km"], df["y_km"], s=sizes, c=pop_vals)

    cbar = plt.colorbar(sc)
    cbar.set_label("Population")
    try:
        cbar.ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
    except Exception:
        pass

    # Draw the core location
    plt.scatter([cx], [cy], s=120, marker="x")

    # Top-3 labels
    top3 = df.nlargest(3, "pop").copy()
    dx = 0.01 * (maxx - minx)
    dy = 0.01 * (maxy - miny)
    for _, row in top3.iterrows():
        label = f"{int(row['pop']):,}"
        plt.text(
            row["x_km"] + dx,
            row["y_km"] + dy,
            label,
            fontsize=8,
            ha="left",
            va="bottom",
            bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7),
        )

    plt.title("Nodes — V1.1 (single core density) — color = population")
    plt.xlabel("x (km)")
    plt.ylabel("y (km)")
    plt.xlim(minx, maxx)
    plt.ylim(miny, maxy)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()


def compute_metrics_hash(metrics: Dict[str, Any]) -> str:
    blob = json.dumps(metrics, sort_keys=True).encode("utf-8")
    return hashlib.sha256(blob).hexdigest()[:16]


def save_artifacts(df: pd.DataFrame, cfg: V11Config, metrics: Dict[str, Any]) -> Dict[str, str]:
    os.makedirs(cfg.out_dir, exist_ok=True)

    nodes_path = os.path.join(cfg.out_dir, "nodes.csv")
    preview_path = os.path.join(cfg.out_dir, "preview.png")
    meta_path = os.path.join(cfg.out_dir, "meta.json")

    df.to_csv(nodes_path, index=False)
    preview_nodes(df, cfg, preview_path)

    meta = {
        "schema_version": cfg.schema_version,
        "dataset_version": cfg.dataset_version,
        "crs": cfg.crs,
        "seed": cfg.seed,
        "generator": {
            "name": "nodes_v1.1_single_core_density",
            "params": {
                "n_nodes": cfg.n_nodes,
                "bbox_km": cfg.bbox_km,
                "core_location": (cfg.core_location or _bbox_center(cfg.bbox_km)),
                "core_sigma_km": cfg.core_sigma_km,
                "density_strength": cfg.density_strength,
                "max_gauss_resamples": cfg.max_gauss_resamples,
                "pop_min": cfg.pop_min,
                "pop_max": cfg.pop_max,
            },
        },
        "region_bbox": list(cfg.bbox_km),
        "metrics": metrics,
        "metrics_hash": compute_metrics_hash(metrics),
        "created_at_utc": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
    }
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

    return {"nodes": nodes_path, "preview": preview_path, "meta": meta_path}


# ------------------------------
# Orchestration
# ------------------------------

def main(cfg: V11Config | None = None) -> pd.DataFrame:
    cfg = cfg or V11Config()
    set_seed(cfg.seed)

    df = generate_nodes_v11(cfg)
    metrics = validate_nodes(df, cfg)
    paths = save_artifacts(df, cfg, metrics)

    print("\n[Nodes V1.1] Build complete:\n" + "-" * 40)
    print(f"Nodes: {len(df)} | bbox: {cfg.bbox_km}")
    print(
        f"Core: {(cfg.core_location or _bbox_center(cfg.bbox_km))}, sigma={cfg.core_sigma_km} km, strength={cfg.density_strength}"
    )
    print(
        f"Mean core dist: {metrics['mean_core_dist_km']:.2f} km (uniform baseline {metrics['baseline_mean_core_dist_km']:.2f} km)"
    )
    print(f"Saved: nodes → {paths['nodes']}\n       preview → {paths['preview']}\n       meta → {paths['meta']}")
    print(f"Metrics hash: {compute_metrics_hash(metrics)}")
    return df


# ------------------------------
# Run
# ------------------------------
_cfg = V11Config(
    seed=50,
    n_nodes=30,
    bbox_km=(0.0, 0.0, 200.0, 200.0),
    core_sigma_km=50.0,
    density_strength=0.8,
    pop_min=1_000,
    pop_max=1_500_000,
    out_dir="maps/sv1.1/dv0.3_v1_1_single_core",
)

_ = main(_cfg)



[Nodes V1.1] Build complete:
----------------------------------------
Nodes: 30 | bbox: (0.0, 0.0, 200.0, 200.0)
Core: (100.0, 100.0), sigma=50.0 km, strength=0.8
Mean core dist: 55.64 km (uniform baseline 77.09 km)
Saved: nodes → maps/sv1.1/dv0.3_v1_1_single_core\nodes.csv
       preview → maps/sv1.1/dv0.3_v1_1_single_core\preview.png
       meta → maps/sv1.1/dv0.3_v1_1_single_core\meta.json
Metrics hash: f4c61e13d021fcec
